In [1]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tslearn.metrics import dtw, soft_dtw, dtw_path
from sklearn.metrics import accuracy_score
from scipy.spatial import distance as dist
from scipy.stats import norm

from reader import get_x_label, get_class_x
from average import amplitude_avg, dtw_avg, soft_dtw_avg

In [50]:
file_name = 'yoga'

In [51]:
class_x = get_class_x(file_name + '_TRAIN')
class_avg = dtw_avg(class_x)

KeyboardInterrupt: 

In [ ]:
dtw_stat = [[list() for i in range(len(class_x[0][0]))] for _ in range(len(class_x))]
for i in range(class_x.shape[0]):
    for j in range(len(class_x[i])):
        path, dist = dtw_path(class_avg[i], class_x[i][j])
        for avg_pos, x_pos in path:
            dtw_stat[i][avg_pos].append(class_x[i][j][x_pos])
dtw_stat = np.array(dtw_stat)
for i in range(dtw_stat.shape[0]):
    for j in range(dtw_stat.shape[1]):
        dtw_stat[i] = np.array(dtw_stat[i])

In [ ]:
dtw_mean = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
dtw_sd = [[list() for i in range(dtw_stat.shape[1])] for _ in range(dtw_stat.shape[0])]
for class_label in range(dtw_stat.shape[0]):
    for point in range(dtw_stat.shape[1]):
        stat = np.array(dtw_stat[class_label][point])
        dtw_mean[class_label][point].append(stat.mean())
        dtw_sd[class_label][point].append(stat.std())

In [ ]:
test_x, test_label = get_x_label(file_name + '_TEST')

In [ ]:
bay_predict = []
dist_predict = []
EPSILON = 10**(-10)
count = 0
for x in test_x:
    min_log_prob = 10**(310)
    min_dist = 100000
    bay_nearest_class = -1
    dist_nearest_class = -1
    for j in range(len(class_avg)):
        # Class j
        path, dist = dtw_path(x, class_avg[j])
        log_prob = 1
        probs = [list() for _ in range(len(x))]
        for x_pos, avg_pos in path:
            area_up = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).cdf(x[x_pos])
            area_low = norm(dtw_mean[j][avg_pos][0], dtw_sd[j][avg_pos][0]).cdf(x[x_pos] - EPSILON)
            prob = area_up - area_low
            if prob < 10**(-310):
                prob = 10**(-310)
            probs[x_pos].append(prob)
        for pos_probs in probs:
#             print(len(pos_probs))
            max_pos_prob = max(pos_probs)
#             print(max_pos_prob)
            log_prob += -np.log(max_pos_prob)
#         print(log_prob)
        if log_prob < min_log_prob:
            min_log_prob = log_prob
            bay_nearest_class = j
        if dist < min_dist:
            min_dist = dist
            dist_nearest_class = j
    bay_predict.append(bay_nearest_class)
    dist_predict.append(dist_nearest_class)
    print(count, ':', bay_nearest_class, dist_nearest_class, test_label[count]-1)
    count += 1

In [ ]:
accuracy_score(test_label-1, bay_predict)